In [1]:
import numpy as np
import pandas as pd
from pprint import pprint
import yfinance as yf


In [2]:

tickers=["AAPL","MSFT","AMZN","GOOGL"]
num_stocks=len(tickers)

cl_price=pd.DataFrame()

In [3]:
for ticker in tickers:
    cl_price[ticker]=yf.download(ticker,period="1y",interval="1d")["Close"]

cl_price.to_csv("Closing_price.csv")

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [4]:

log_return = np.log(1+cl_price.pct_change())
print(log_return)

returns=cl_price.pct_change().dropna()

random_weights = np.array(np.random.random(num_stocks))
print(random_weights)

rebalance_weights = random_weights / np.sum(random_weights)
print(rebalance_weights)

exp_ret = np.sum((log_return.mean() * rebalance_weights) * 252)
print(exp_ret)

exp_vol = np.sqrt(
np.dot(
    rebalance_weights.T,
    np.dot(
        log_return.cov() * 252,
        rebalance_weights
    )
)
)
print(exp_vol)

sharpe_ratio = exp_ret / exp_vol



                AAPL      MSFT      AMZN     GOOGL
Date                                              
2022-10-06       NaN       NaN       NaN       NaN
2022-10-07 -0.037410 -0.052191 -0.048890 -0.027388
2022-10-10  0.002353 -0.021533 -0.007799 -0.008344
2022-10-11 -0.010308 -0.016892 -0.012927 -0.006973
2022-10-12 -0.004616  0.001507  0.006130  0.003903
...              ...       ...       ...       ...
2023-09-29  0.003042  0.006705  0.009008 -0.011020
2023-10-02  0.014727  0.018979  0.018240  0.024980
2023-10-03 -0.007800 -0.026482 -0.037301 -0.013053
2023-10-04  0.007282  0.017617  0.018116  0.020997
2023-10-05  0.007172  0.001253 -0.008223 -0.001258

[251 rows x 4 columns]
[0.44179836 0.62812274 0.24964511 0.05858868]
[0.32057235 0.45577079 0.18114445 0.0425124 ]
0.19874572270574653
0.26959600570410036


In [5]:

num_of_portfolios = 5000  #number of simulations
all_weights = np.zeros((num_of_portfolios, num_stocks))
ret_arr = np.zeros(num_of_portfolios)
vol_arr = np.zeros(num_of_portfolios)
sharpe_arr = np.zeros(num_of_portfolios)

for ind in range(num_of_portfolios):
    weights = np.array(np.random.random(num_stocks))
    weights = weights / np.sum(weights)
    all_weights[ind, :] = weights
    ret_arr[ind] = np.sum((returns.mean() * weights)*252)
    vol_arr[ind] = np.sqrt(
        np.dot(weights.T, np.dot(log_return.cov() * 252, weights))
    )
    sharpe_arr[ind] = ret_arr[ind]/vol_arr[ind]

simulations_data = [ret_arr, vol_arr, sharpe_arr, all_weights]
simulations_df = pd.DataFrame(data=simulations_data).T
simulations_df.columns = [
    'Returns',
    'Volatility',
    'Sharpe Ratio',
    'Portfolio Weights'
]
simulations_df = simulations_df.infer_objects()
print('SIMULATIONS RESULT:')
print(simulations_df.to_markdown)

SIMULATIONS RESULT:
<bound method DataFrame.to_markdown of        Returns  Volatility  Sharpe Ratio  \
0     0.243190    0.273625      0.888770   
1     0.284858    0.275657      1.033378   
2     0.210302    0.269386      0.780673   
3     0.285308    0.273384      1.043616   
4     0.258893    0.279208      0.927241   
...        ...         ...           ...   
4995  0.251345    0.296929      0.846482   
4996  0.209631    0.274571      0.763484   
4997  0.233243    0.300399      0.776444   
4998  0.260191    0.284249      0.915363   
4999  0.251429    0.270444      0.929687   

                                      Portfolio Weights  
0     [0.37028088545962157, 0.10589631265237075, 0.2...  
1     [0.2356557160525137, 0.3172921276593095, 0.086...  
2     [0.5093002053701344, 0.09018852063583524, 0.30...  
3     [0.3424343443194496, 0.1941585594828521, 0.048...  
4     [0.274656878530648, 0.1441959810382811, 0.2113...  
...                                                 ...  
4995  

In [6]:

max_sharpe_ratio = simulations_df.loc[simulations_df['Sharpe Ratio'].idxmax()]
min_volatility = simulations_df.loc[simulations_df['Volatility'].idxmin()]
print('MAX SHARPE RATIO:')
print(max_sharpe_ratio)
print('MIN VOLATILITY:')
print(min_volatility)


MAX SHARPE RATIO:
Returns                                                       0.310903
Volatility                                                    0.283899
Sharpe Ratio                                                   1.09512
Portfolio Weights    [0.14207638749091428, 0.4023563458683381, 0.00...
Name: 2696, dtype: object
MIN VOLATILITY:
Returns                                                       0.245318
Volatility                                                    0.259347
Sharpe Ratio                                                  0.945909
Portfolio Weights    [0.5909788900530164, 0.24833435920961433, 0.08...
Name: 866, dtype: object
